In [168]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install psycopg2

In [169]:
from psycopg2 import connect
import pandas as pd

In [170]:
conn = connect(host='localhost', dbname='StoragesDB')

In [171]:
conn

<connection object at 0x0000021E8275A598; dsn: 'host=localhost dbname=StoragesDB', closed: 0>

In [172]:
D_START = '01-01-2001'
D_END = '09-03-2020'
query = f"""
    SELECT recept.ddate, sum(recgoods.volume * recgoods.price) AS sm,
    goods.g_group FROM goods
    JOIN recgoods ON (recgoods.goods = goods.id)
    JOIN recept ON (recept.id = recgoods.id)
    WHERE
    (recept.ddate >= '{D_START}') AND (recept.ddate <= '{D_END}')
    GROUP BY
    recept.ddate, goods.g_group
    ORDER BY goods.g_group, recept.ddate;
"""

In [173]:
query

"\n    SELECT recept.ddate, sum(recgoods.volume * recgoods.price) AS sm,\n    goods.g_group FROM goods\n    JOIN recgoods ON (recgoods.goods = goods.id)\n    JOIN recept ON (recept.id = recgoods.id)\n    WHERE\n    (recept.ddate >= '01-01-2001') AND (recept.ddate <= '09-03-2020')\n    GROUP BY\n    recept.ddate, goods.g_group\n    ORDER BY goods.g_group, recept.ddate;\n"

In [174]:
with conn.cursor() as cur:
    cur.execute(query)
    for row in cur:
        print(row)

(datetime.date(2020, 3, 7), 125000.0, 1)
(datetime.date(2020, 3, 8), 30000.0, 1)
(datetime.date(2020, 3, 9), 50000.0, 1)
(datetime.date(2020, 3, 7), 52500.0, 2)
(datetime.date(2020, 3, 8), 50000.0, 3)
(datetime.date(2020, 3, 9), 10000.0, 3)


In [175]:
ALPHA = 0.9
result_data = pd.DataFrame(columns=['goods_group','date', 'predict_val', 'error'])

In [176]:
with conn.cursor() as cur:
    cur.execute(query)
    
    def compute_predict(cur, prev_group, prev_predict, pdframe):
        row = cur.fetchone()
        if row is None:
            return pdframe
        
        dt, sm, group = row
        if (prev_group is None) or (prev_group != group):
            predict_val = ALPHA*sm
        else:
            predict_val = ALPHA*sm+(1-ALPHA)*prev_predict
            
        pdframe = pdframe.append({'goods_group':group, 'date':dt,'predict_val':predict_val,
                        'error':abs(sm-predict_val)}, ignore_index=True)

        pdframe = compute_predict(cur, group, predict_val, pdframe)
        
        return pdframe
        
    result_data = compute_predict(cur, None, None, result_data) 
    
conn.close()

In [177]:
result_data

,goods_group,date,predict_val,error
0,1,2020-03-07,112500.0,12500.0
1,1,2020-03-08,38250.0,8250.0
2,1,2020-03-09,48825.0,1175.0
3,2,2020-03-07,47250.0,5250.0
4,3,2020-03-08,45000.0,5000.0
5,3,2020-03-09,13500.0,3500.0
